# Extension 9

In this section, we explore how we can make the code much more efficient by using pre-computed values and a prime sieve. 

In [1]:
# Imports
import math as m
import numpy as np
import matplotlib.pyplot as plt 

# Set default values for K, I, and the maximum n value - these will be explained later
K_norm = 30

I_norm = 1e9

n_max = 20000

The function `prime_sieve` helps to find all the prime numbers up to a number $n$. This function is an efficient and fast way to obtain the primes especially for large $n$.


How the Prime Sieve algortihm works in theory:


**1.** Creates a list of consecutive integers from $2$ to $n$, the list starts with $2$ and marks all the multiples of that number (but not itself) as composite.


**2.** It then moves onto the next smallest unmarked number($3..5..7$) and does the same as before, repeating itself until it hits $\sqrt{n}$ and the rest of the unmarked numbers in the list are prime.


`prime_sieve` helps to make the `get_prime_factors` function and , thus, the `s(n)` function quicker.

In [2]:
# Prime sieve is an efficient way to find all the primes up to (and including) n
def prime_sieve(maximum=n_max):

    # There are no primes less than 2
    if maximum < 2:
        return None

    # Will create a list of True or False values corresponding to whether the index is a prime number
    # e.g. 3 is a prime number so the value in the 3rd index (4th position) will be true by the end of the function
    prime_mask = [True for _ in range(maximum+1)]

    # 0 and 1 are not prime
    prime_mask[0], prime_mask[1] = False, False

    # Only need to check up to root n
    for p in range(2, int(m.sqrt(maximum)) + 1):
        if prime_mask[p]:
            # go over multiples of each prime number, which will not be prime, and so set value in prime_mask to False
            for i in range(p*p, maximum+1, p):
                prime_mask[i] = False

    # change prime_mask into a numpy array so we can use a boolean mask
    prime_mask = np.array(prime_mask)

    # generate the array of numbers up to n and apply boolean mask to create array of prime numbers
    return np.array([i for i in range(maximum+1)])[prime_mask]


# PRIMES calculated using prime sieve up to the largest n we are going up to, divided by 2
# We can stop here as n cannot have a prime factor larger than n/2
# this can then be used to calculate s(n) for ALL the ns up to n_max, instead of prime_sieve being called for every n
PRIMES = prime_sieve(int(n_max/2) + 1)

A few things to note which make this much more efficient than before:

- It only searches up to $\sqrt{n}$, because any non-primes bigger than this will be caught by multiples of numbers up to $\sqrt{n}$

- Since it sets values in prime mask to False as it goes along, it ends up skipping all numbers in the original interval which aren't prime, since they will have been caught and set to False by one of their prime factors earlier in the loop

- By using a boolean mask we can edit our original array in place without having to make a seperate list storing which numbers are prime

- When the list of primes is caluclated at the very end, we only called the `prime sieve` function up to $\sqrt{n}$, since there can be up to 1 prime factor of a number $n$ thats bigger than $\sqrt{n}$

## Calculating $s(n)$

We can now use the **prime factors** to jump straight to $s(n)$ without having to calculate all the individual factors of $n$, which was very lengthy.

We can use the **prime factors** to find the **sum of divisors** for a given $n$ using the formula:

$$
\sigma(n) = \prod_{i=1}^{k} \left( \frac{p_i^{a_i + 1} - 1}{p_i - 1} \right)
$$

Where $p_i$ are the prime factors and $a_i$ are the powers of the prime factors.

The **sum of proper divisors** is then calculated using the forumla:

$$
s(n) = \sigma(n) - n
$$

First, we need to make a function which uses the primes calculated using `prime_sieve` to find the prime factors of a number:

In [3]:
# Using the PRIMES list get the prime factors of a number n
def get_prime_factors(n):

    primes = PRIMES

    # create an empty dictionary of prime factors
    # we will store this is prime : exponent pairs
    # e.g. n=12 will have prime_factors = {2 : 2, 3 : 1} as 12 = 2^2 x 3
    prime_factors = {}

    # go through each prime to check if it's a divisor of n
    for prime in primes:

        # Only need to check primes up to root n
        # There may be 1 prime bigger than root n, we will deal with this at the end
        if prime**2 > n:
            break

        # check if prime divides n and add to dictionary if it does
        if n % prime == 0:
            prime_factors[int(prime)] = 1

            # now we set n to n / prime so we dont count the same prime factor more than once
            n = n / prime

            # check if this new n is still divisible by the same prime.
            # For as long as it works keep increasing the exponent value assigned to the prime, and dividing n
            while n % prime == 0:
                prime_factors[int(prime)] += 1
                n = n / prime

        # if n reaches 1 we can stop early as we know we have found the prime factorisation of n
        if n == 1:
            break

    # now we check for the potential prime factor bigger than root n.
    # If we have checked all the primes up to root n and still haven't found all the prime factors,
    # we must be left with the last prime factor as n
    if n > 1:
        prime_factors[int(n)] = 1

    # return our completed list of prime factors
    return prime_factors


Now we are ready to calculate $s(n)$:

In [4]:
# Calculate s(n)
def s(n):

    # 1 has no proper divisors
    if n == 1:
        return 0

    total = 1

    # stores keys and values in unpackable pairs
    primes = get_prime_factors(n).items()

    for prime, power in primes:
        total *= (prime**(power+1) - 1)/(prime - 1)

    # return s(n)
    return total - n

## How much has this improved our code efficiency?

Previously, the program took 10.5 seconds to run for $i = 10^{9}$, $k = 30$ and caluclating and clasifying all sequences up to $n = 20000$. With these updates, it takes 4.87 seconds.

For bigger values of $i, k$ and $n$, for example $i = 10^{10}$, $k = 50$ and all sequences up to $n = 30000$, the program previously took 101.03 seconds. With these updates, it takes just 17.52 seconds.

Clearly these are massive imporvements in the speed. But we can do better...

## Where should we focus our efforts to further increase code efficiency? ##
In order to find the parts of code which are slowing us down, we can use the `time` module in python. When running the code for the second(larger) set of values above, the corresponding times were calculated:

- Running the prime sieve function (which only gets called once) for $n = 30000$ took less than 0.005 seconds. Since this fucntion does not take $K$ or $I$ as inputs, it will also only get slower as $n$ increases.
- On the other hand, generating the 30000 aliquot sequences took 15.82 seconds. This time will also increase with all 3 of $n$, $I$ and $K$.

Therefore it is clear that the latter is the part where we should focus our efforts to make the program faster.

## How do we make it faster? ##
At the moment, every time `s(n)` is called the program completely regenerates its prime factors and then sums them. We can use the decorator `@functools.cache` to make the program much faster. This works by storing the values returned by a function, and then if it is called again with the same input it will straight away return this value again without re-running the function.

Even though the function which generates the aliquot sequence will take the longest (since it contains multiple calls of `s(n)`), we cannot cache it since we will never call it with the same input more than once. On the other hand `s(n)` will be called with the same input many times, especially for large $n$. Caching `s(n)` will be more effective than the `get_prime_factors` function, since a call of `s(n)` contains a call of `get_prime_factors`. This means caching `s(n)` will have the effect of caching `get_prime_factors` as well as skipping unecessary use of the $s(n)$ formula.

Implementation of this is very simple, we simply add the decorator right above the function:

In [5]:
import functools

@functools.cache
def s(n):

    # 1 has no proper divisors
    if n == 1:
        return 0

    # the following uses the formula: (insert in latex later)
    # to calculate the sum of all the factors of n using only its prime factors
    total = 1

    primes = get_prime_factors(n).items()

    for prime, power in primes:
        total *= (prime**(power+1) - 1)/(prime - 1)

    # return s(n)
    return total - n

## How much has this improved our code efficiency?
Generating and sorting the first 30000 aliquot sequences now takes just 2.44 seconds with $K = 50$ and $I = 10^9$. Running the full code from start to finish now takes just over 2.5 seconds to run (a dramatic improvement from the previous 17.5 seconds)! Too better compare this to later results, we also run the program for $n = 100000$, $K = 1000$, $I = 10^{12}$. This takes about 21.5 seconds to run in full.

There is one more thing we can do to try and make the code faster. Currently the `get_prime_factors` function completely re-computes the prime factors for each new n. But the way we have programmed it naturally lends itself to a simple change which can make it faster. Since the function works by continuously dividing n until it reaches 1, if we find a number we have already factorised during the dividing process we straight away know the factorisation of the original number. For example, lets say we know $15 = 3 * 5$. Then if we have $n = 60$, first the function will divide by 2 twice. We will then reach $n = \frac{60}{4} = 15$ Therefore we straight away know that the prime factors of 60 are $2^2 * 15 = 2^2 * 3 * 5$

We implement this by adding a few bits of code:

- First we make an empty dictionary called `factorisations`. This will store the prime factors of a number in key, value pairs where the values are dictionaries themselves.

- Then we adjust `get_prime_factors` as follows:

In [ ]:
# Using the PRIMES list get the prime factors of a number n
def get_prime_factors(n):

    global factorisations

    if n in factorisations:
        return factorisations[n]

    primes = PRIMES

    # create an empty dictionary of prime factors
    # we will store this is prime : exponent pairs
    # e.g. n=12 will have prime_factors = {2 : 2, 3 : 1} as 12 = 2^2 x 3
    prime_factors = {}

    # go through each prime to check if it's a divisor of n
    for prime in primes:

        # Only need to check primes up to root n
        # There may be 1 prime bigger than root n,we will deal with this at the end
        if prime**2 > n:
            break

        # check if prime divides n and add to dictionary if it does
        if n % prime == 0:
            prime_factors[int(prime)] = 1

            # now we set n to n / prime, so we don't count the same prime factor more than once
            n = n / prime

            # New code starts here
            #Check if we know the factorisation of the new n we created
            if n in factorisations:
                facts = factorisations[n].copy()

                # Combine the factors we already had with the new ones
                for key, power in prime_factors.items():
                    try:
                        facts[key] += power

                    except KeyError:
                        facts[key] = power

                # We have found the full factorisation so we can end the program early
                return facts
            # And ends here

            # check if this new n is still divisible by the same prime.
            # For as long as it works keep increasing the exponent value assigned to the prime, and dividing n
            while n % prime == 0:
                prime_factors[int(prime)] += 1
                n = n / prime

        # if n reaches 1 we can stop early as we know we have found the prime factorisation of n
        if n == 1:
            break

    # now we check for the potential prime factor bigger than root n.
    # If we have checked all the primes up to root n and still haven't found all the prime factors,
    # we must be left with the last prime factor as n
    if n > 1:
        prime_factors[int(n)] = 1

    # return our completed list of prime factors
    return prime_factors

- Lastly, after we have called `get_prime_factors` inside `s(n)`, we simply add the output to our factorisations dictionary. (The complete uopdated code is included at the end of this section)

## How much has this improved our code efficiency?
For the values $n = 100000$, $K = 1000$ and $I = 10^{12}$, the program now takes about 19.5 seconds. This has made the program faster but not nearly as effectively as the previous changes. After running some simple tests, it looks like the new code is being run on about half of the calls to `get_prime_factors`. This means the time savings are probably only slight due to the code taking a long time to execute - this is believable as the dictioary will be very big and so will take a long time to search through, and merging the dictionaries is slow. 

The full final code has been included below:

In [1]:
# Imports
import functools
import math as m
import numpy as np
import time as t

# Set default values for K, I, and the maximum n value
K_norm = 1000

I_norm = 1e12

n_max = 100000

start_time = t.perf_counter()


def prime_sieve(maximum=n_max):

    # There are no primes less than 2
    if maximum < 2:
        return None

    # Will create a list of True or False values corresponding to whether the index is a prime number
    # e.g. 3 is a prime number so the value in the 3rd index (4th position) will be true by the end of the function
    prime_mask = [True for _ in range(maximum+1)]

    # 0 and 1 are not prime
    prime_mask[0], prime_mask[1] = False, False

    # Only need to check up to root n
    for p in range(2, int(m.sqrt(maximum)) + 1):
        if prime_mask[p]:
            # go over multiples of each prime number, which will not be prime, and so set value in prime_mask to False
            for i in range(p*p, maximum+1, p):
                prime_mask[i] = False

    # generate the array of numbers up to n and apply boolean mask to create array of prime numbers
    return np.array([i for i in range(maximum+1)])[prime_mask]


PRIMES = prime_sieve(int(n_max/2) + 1)
print(f'primes found after {t.perf_counter() - start_time}s')
factorisations = {}


# Using the PRIMES list get the prime factors of a number n
def get_prime_factors(n):

    global factorisations

    if n in factorisations:
        return factorisations[n]

    primes = PRIMES

    # create an empty dictionary of prime factors
    # we will store this is prime : exponent pairs
    # e.g. n=12 will have prime_factors = {2 : 2, 3 : 1} as 12 = 2^2 x 3
    prime_factors = {}

    # go through each prime to check if it's a divisor of n
    for prime in primes:

        # Only need to check primes up to root n
        # There may be 1 prime bigger than root n,we will deal with this at the end
        if prime**2 > n:
            break

        # check if prime divides n and add to dictionary if it does
        if n % prime == 0:
            prime_factors[int(prime)] = 1

            # now we set n to n / prime, so we don't count the same prime factor more than once
            n = n / prime

            if n in factorisations:
                facts = factorisations[n].copy()

                for key, power in prime_factors.items():
                    try:
                        facts[key] += power

                    except KeyError:
                        facts[key] = power

                return facts

            # check if this new n is still divisible by the same prime.
            # For as long as it works keep increasing the exponent value assigned to the prime, and dividing n
            while n % prime == 0:
                prime_factors[int(prime)] += 1
                n = n / prime

        # if n reaches 1 we can stop early as we know we have found the prime factorisation of n
        if n == 1:
            break

    # now we check for the potential prime factor bigger than root n.
    # If we have checked all the primes up to root n and still haven't found all the prime factors,
    # we must be left with the last prime factor as n
    if n > 1:
        prime_factors[int(n)] = 1

    # return our completed list of prime factors
    return prime_factors

# Calculate s(n)
@functools.cache
def s(n):

    global factorisations

    # 1 has no proper divisors
    if n == 1:
        return 0

    # the following uses the formula: (insert in latex later)
    # to calculate the sum of all the factors of n using only its prime factors
    total = 1

    primes_dict = get_prime_factors(n)
    primes = primes_dict.items()
    factorisations[n] = primes_dict

    for prime, power in primes:
        total *= (prime**(power+1) - 1)/(prime - 1)

    # return s(n)
    return total - n


# Calculate the aliquot sequence of n
def aliq_seq(n, K=K_norm, I=I_norm):

    # n will be the first element of its aliquot sequence
    seq = [n]

    # make an empty set to record all the s(n)s in order to detect loops
    seen = set()

    curr_n = n
    # loop max k times to compute and check each s(n)
    # return always ends the function so no more code will run after one of the checks is 'failed'
    for k in range(K):

        # check if sequence is terminated
        if curr_n == 0:
            return seq, 'terminated'

        # check if number is bigger than specified i
        if curr_n >= I:
            return seq, 'I reached'

        # check if sequence has looped
        if curr_n in seen:
            return seq, 'looped'

        # add n to sequence of seen numbers
        # has to be done after checks otherwise it will always think it loops
        seen.add(curr_n)

        # calculate next term of sequence if n has 'passed' all tests
        curr_n = s(curr_n)
        seq.append(int(curr_n))

    # If loop finished and has computed all k s(n), the sequence is finished (up to where we have decided to stop)
    # so returns sequence and the status that k has been reached
    return seq, 'K reached'


# make dictionary to count the number of types of each sequence
counts = {'terminated': 0, 'looped': 0, 'I reached': 0, 'K reached': 0}

# generate sequences up to 20000 (or whatever n_max is) and update counts dictionary
for i in range(1, n_max + 1):
    status = aliq_seq(i)[1]
    counts[status] += 1

print(f'aliquot sequences found after {t.perf_counter() - start_time}s')
print(counts)

primes found after 0.01881037198472768s
aliquot sequences found after 20.909886535955593s
{'terminated': 82986, 'looped': 2085, 'I reached': 14929, 'K reached': 0}
